In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [3]:
from config import DATA_DIR
ml_file = PROJECT_ROOT / DATA_DIR / "processed" / "features_for_ml.csv"

In [5]:
ml_df = pd.read_csv(ml_file, parse_dates=["Date"])

In [6]:
ml_df.describe()

,Date,rolling_14d_adjusted,price_available,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,wind_speed_10m_max,et0_fao_evapotranspiration,temperature_2m_max_30d_lag,...,et0_fao_evapotranspiration_120d_lag,et0_fao_evapotranspiration_180d_lag,heatwave_flag,coldwave_flag,heavy_rain_flag,temp_missing,USD_TL,trend,trends_missing,is_trend_zero
count,8769,8769.000000,8769.0,8769.000000,8769.000000,8769.000000,8769.000000,8769.000000,8769.000000,8769.000000,...,8769.000000,8769.000000,8769.000000,8769.000000,8769.000000,8769.0,8769.000000,4776.000000,4776.0,4776.000000
mean,2012-08-28 23:45:23.092712960,2.829628,1.0,17.114084,10.531144,3.428692,3.099704,13.969997,2.386388,17.280467,...,2.483747,2.440992,0.000114,0.057247,0.000798,0.0,6.794190,66.270472,0.0,0.175042
min,1999-12-26 00:00:00,0.730755,1.0,-2.800000,-10.800000,0.000000,0.000000,4.300000,0.180000,3.850000,...,1.011083,1.246722,0.000000,0.000000,0.000000,0.0,0.535502,0.000000,0.0,0.000000
25%,2005-12-26 00:00:00,2.192070,1.0,12.000000,5.400000,0.000000,0.000000,10.700000,1.270000,12.293333,...,1.618333,1.742167,0.000000,0.000000,0.000000,0.0,1.424333,27.807069,0.0,0.000000
50%,2012-04-06 00:00:00,2.764137,1.0,17.600000,10.400000,0.300000,0.300000,13.000000,2.160000,16.973333,...,2.464417,2.456500,0.000000,0.000000,0.000000,0.0,1.791467,49.184661,0.0,0.000000
75%,2019-11-29 00:00:00,3.363210,1.0,22.800000,16.300000,4.000000,3.500000,16.200000,3.420000,22.840000,...,3.339083,3.109167,0.000000,0.000000,0.000000,0.0,5.953000,83.319809,0.0,0.000000
max,2025-12-31 00:00:00,7.769791,1.0,35.800000,25.500000,78.700000,78.700000,53.700000,6.740000,29.780000,...,4.290583,3.861833,1.000000,1.000000,1.000000,0.0,42.939500,1087.689728,0.0,1.000000
std,NaN,1.207779,0.0,6.963901,6.844228,6.422489,6.017589,4.789735,1.336889,6.138736,...,0.893603,0.705018,0.010679,0.232327,0.028244,0.0,10.445633,68.401890,0.0,0.380043


In [7]:
ml_df.columns

Index(['Date', 'rolling_14d_adjusted', 'price_available', 'temperature_2m_max',
       'temperature_2m_min', 'precipitation_sum', 'rain_sum',
       'wind_speed_10m_max', 'et0_fao_evapotranspiration',
       'temperature_2m_max_30d_lag', 'temperature_2m_max_60d_lag',
       'temperature_2m_max_90d_lag', 'temperature_2m_max_120d_lag',
       'temperature_2m_max_180d_lag', 'temperature_2m_min_30d_lag',
       'temperature_2m_min_60d_lag', 'temperature_2m_min_90d_lag',
       'temperature_2m_min_120d_lag', 'temperature_2m_min_180d_lag',
       'precipitation_sum_30d_lag', 'precipitation_sum_60d_lag',
       'precipitation_sum_90d_lag', 'precipitation_sum_120d_lag',
       'precipitation_sum_180d_lag', 'rain_sum_30d_lag', 'rain_sum_60d_lag',
       'rain_sum_90d_lag', 'rain_sum_120d_lag', 'rain_sum_180d_lag',
       'wind_speed_10m_max_30d_lag', 'wind_speed_10m_max_60d_lag',
       'wind_speed_10m_max_90d_lag', 'wind_speed_10m_max_120d_lag',
       'wind_speed_10m_max_180d_lag', 'et0_fao_e

In [8]:
target_col = 'rolling_14d_adjusted'

drop_cols = [
    'Date',
    target_col,
    'trend',
    'trends_missing',
    'is_trend_zero'
]

In [9]:
X = ml_df.drop(columns=drop_cols)
y = ml_df[target_col]

In [10]:
train_mask = ml_df['Date'] < '2019-01-01'

X_train = X.loc[train_mask]
y_train = y.loc[train_mask]

X_test = X.loc[~train_mask]
y_test = y.loc[~train_mask]

In [11]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6318, 46) (2451, 46) (6318,) (2451,)


In [15]:
ridge_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', Ridge(alpha=1.0))
])

In [16]:
ridge_pipe.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('scaler', ...), ('ridge', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"copy copy: bool, default=TrueIf False, try to avoid a copy and do inplace scaling instead.This is not guaranteed to always work inplace; e.g. if the data isnot a NumPy array or scipy.sparse CSR matrix, a copy may still bereturned.",True
,"with_mean with_mean: bool, default=TrueIf True, center the data before scaling.This does not work (and will raise an exception) when attempted onsparse matrices, because centering them entails building a densematrix which in common use cases is likely to be too large to fit inmemory.",True
,"with_std with_std: bool, default=TrueIf True, scale the data to unit variance (or equivalently,unit standard deviation).",True
,"alpha alpha: {float, ndarray of shape (n_targets,)}, default=1.0Constant that multiplies the L2 term, controlling regularizationstrength. `alpha` must be a non-negative float i.e. in `[0, inf)`.When `alpha = 0`, the objective is equivalent to ordinary leastsquares, solved by the :class:`LinearRegression` object. For numericalreasons, using `alpha = 0` with the `Ridge` object is not advised.Instead, you should use the :class:`LinearRegression` object.If an array is passed, penalties are assumed to be specific to thetargets. Hence they must correspond in number.",1.0
,"fit_intercept fit_intercept: bool, default=TrueWhether to fit the intercept for this model. If setto false, no intercept will be used in calculations(i.e. ``X`` and ``y`` are expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"max_iter max_iter: int, default=NoneMaximum number of iterations for conjugate gradient solver.For 'sparse_cg' and 'lsqr' solvers, the default value is determinedby scipy.sparse.linalg. For 'sag' solver, the default value is 1000.For 'lbfgs' solver, the default value is 15000.",None


In [17]:
y_pred_ridge = ridge_pipe.predict(X_test)

In [18]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred_ridge))
mae = mean_absolute_error(y_test, y_pred_ridge)
r2 = r2_score(y_test, y_pred_ridge)

print("Ridge Performance")
print("RMSE:", rmse)
print("MAE:", mae)
print("R²:", r2)

Ridge Performance
RMSE: 9.210608078945292
MAE: 7.346587296557428
R²: -84.319683792233


In [19]:
coef = (
    pd.Series(
        ridge_pipe.named_steps['ridge'].coef_,
        index=X.columns
    )
    .sort_values()
)

coef

temperature_2m_min_180d_lag           -3.717747
temperature_2m_max_120d_lag           -1.349380
temperature_2m_max_60d_lag            -1.332526
temperature_2m_min_90d_lag            -1.296169
et0_fao_evapotranspiration_90d_lag    -0.548508
precipitation_sum_180d_lag            -0.519978
et0_fao_evapotranspiration_180d_lag   -0.404907
precipitation_sum_60d_lag             -0.374310
precipitation_sum_120d_lag            -0.279376
precipitation_sum_30d_lag             -0.244982
et0_fao_evapotranspiration_30d_lag    -0.184100
temperature_2m_max_30d_lag            -0.183395
temperature_2m_min                    -0.131322
season_development                    -0.065824
coldwave_flag                         -0.065150
precipitation_sum                     -0.038263
heavy_rain_flag                       -0.036389
season_harvest                        -0.024534
temperature_2m_max                    -0.021766
et0_fao_evapotranspiration            -0.011481
heatwave_flag                          0